# To Do
- 維運機制
- 測試 session.get 是否仍有描述限制
    - 不行的話就改proxy
    - https://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY.html

# 環境設定

## install packages

In [71]:
# !pip install twstock
# !pip install lxml
# !pip install pandas
# !pip install bs4
# !pip install pystan==2.19.1.1
# !pip install convertdate
# !pip install prophet
# !pip install plotly 

## import packages

In [1]:
import re
import twstock
import time
import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3
from prophet import Prophet
from get_top100_stock import get_top100_stock

# 找出交易量Top100的股票清單

In [2]:
url = 'https://tw.stock.yahoo.com/rank/volume'
resp = requests.get(url)
soup = BeautifulSoup(resp.text)

top100_stocks = []
for i in soup.findAll('li', {'class':'List(n)'}):
    top100_stocks.append([i.select('div > div')[1].text, # 名次
                             i.select('div > div')[4].text, # 股名
                             i.select('div > div')[5].text, # 股號
#                          i.select('div > div')[6].text, # 成交價 
#                          i.select('div > div')[9].text,  # 最高
#                          i.select('div > div')[10].text, # 最低
#                          i.select('div > div')[11].text, # 價差
                        i.select('div > div')[12].text, # 成交量(張)
                        i.select('div > div')[13].text # 成交值(億)
                             ])
top100_stocks = pd.DataFrame(top100_stocks,
                             columns=['rank', 'name', 'no', 'transaction', 'turnover'])
top100_stocks['no'] = top100_stocks['no'].apply(lambda x: re.sub(r'.TW', '', x))
top100_stocks.head()

,rank,name,no,transaction,turnover
0,1,長榮,2603,"692,422",638.4318
1,2,陽明,2609,"485,764",519.6718
2,3,群創,3481,"397,107",88.9949
3,4,中鋼,2002,"396,000",145.1669
4,5,華航,2610,"369,381",66.7941


# 代理伺服器

## 取得代理伺服器清單

In [3]:
# def GetProxyList(counts):
#     url = 'https://www.us-proxy.org/'
#     resp = requests.get(url)
#     soup = BeautifulSoup(resp.text)
#     proxies = []

#     for proxy in soup.find('tbody').findAll('tr'):
#         proxies.append([td.text for td in proxy.findAll('td')])
#     proxies = pd.DataFrame(proxies, columns = ['IPADDRESS','PORT','CODE','COUNTRY','ANONYMITY','GOOGLE','HTTPS','LASTCHECKED'])
#     return proxies

In [4]:
# ProxyList = GetProxyList(counts=50)
# ProxyList

## 透過代理伺服器請求資料

In [5]:
# proxiex = {'http':'http://118.163.94.3:80',
#            'https':'https://106.104.148.208:80'} 
# url = 'https://www.twse.com.tw/en/exchangeReport/STOCK_DAY?response=json&date={}&stockNo={}'.format('202105', '2330')
# rs = requests.sessions.session()
# data = rs.get(url, proxies=proxy,timeout = 10)
# data.json()

# 讀取Top100近兩年資料

In [6]:
def get_l24m_stockinfo(stockid, start, end):
    yms =pd.date_range(start=start, end=end, freq='MS')
    df = [] 
    n = 0
    break_times = 0
    while n < len(yms):
        ym = yms[n]
        if n % 6 ==0:
            rs = requests.session()
            # Sleep 15 seconds, and then create a new session.
            time.sleep(15)
        try:
            url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={}&stockNo={}'.format(ym.strftime('%Y%m%d'), str(stockid))
            print(url)
            data = rs.get(url)
            ndf = pd.DataFrame(data=data.json()['data'], columns=data.json()['fields'])
            ndf['title'] = data.json()['title']
            df.append(ndf)
            n += 1
            break_times = 0
        except:
            break_times +=1
            if break_times >= 10:
                break
            print('ERROR and Retry:', stockid)
            
    df = pd.concat(df, ignore_index=True)
    
    df['STOCKID'] = df['title'].apply(lambda x: x.split(' ', -1)[1])
    df['STOCKNAME'] = df['title'].apply(lambda x: x.split(' ', -1)[2])
    df['日期'] = df['日期'].apply(lambda x: x.replace(x[:3], str(int(x[:3])+1911)))
    df = df.loc[:,['STOCKID', 'STOCKNAME', '日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']]
    df = df.rename(columns={'日期': 'DATE',
                            '成交股數': 'TRADEVOLUME',
                            '成交金額': 'TRADEVALUE',
                            '開盤價':'OPENINGPRICE',
                            '最高價':'HIGHESTPRICE',
                            '最低價':'LOWESTPRICE',
                            '收盤價':'CLOSINGPRICE',
                            '漲跌價差':'CHANGE',
                            '成交筆數':'TRANSACTION'})
    df['UPDATETIME'] = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    return df

In [7]:
df = get_l24m_stockinfo(stockid=2330, start='2021-01-01', end=datetime.datetime.now().strftime('%Y-%m-%d'))
df

https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20210101&stockNo=2330
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20210201&stockNo=2330
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20210301&stockNo=2330
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20210401&stockNo=2330
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20210501&stockNo=2330


,STOCKID,STOCKNAME,DATE,TRADEVOLUME,TRADEVALUE,OPENINGPRICE,HIGHESTPRICE,LOWESTPRICE,CLOSINGPRICE,CHANGE,TRANSACTION,UPDATETIME
0,2330,台積電,2021/01/04,"39,489,959","21,127,581,248",530.00,540.00,528.00,536.00,+6.00,"33,316",2021-05-29 20:02:13
1,2330,台積電,2021/01/05,"34,839,391","18,761,831,567",536.00,542.00,535.00,542.00,+6.00,"28,512",2021-05-29 20:02:13
2,2330,台積電,2021/01/06,"55,614,434","30,572,783,229",555.00,555.00,541.00,549.00,+7.00,"55,462",2021-05-29 20:02:13
3,2330,台積電,2021/01/07,"53,392,763","30,018,630,685",554.00,570.00,553.00,565.00,+16.00,"47,905",2021-05-29 20:02:13
4,2330,台積電,2021/01/08,"62,957,148","36,339,702,855",580.00,580.00,571.00,580.00,+15.00,"56,426",2021-05-29 20:02:13
...,...,...,...,...,...,...,...,...,...,...,...,...
89,2330,台積電,2021/05/24,"15,981,036","9,090,836,697",570.00,572.00,566.00,568.00,-5.00,"20,189",2021-05-29 20:02:13
90,2330,台積電,2021/05/25,"35,445,350","20,525,932,956",576.00,584.00,573.00,583.00,+15.00,"39,032",2021-05-29 20:02:13
91,2330,台積電,2021/05/26,"19,555,305","11,433,686,898",587.00,588.00,581.00,585.00,+2.00,"21,034",2021-05-29 20:02:13
92,2330,台積電,2021/05/27,"70,061,002","40,643,804,852",580.00,582.00,573.00,582.00,-3.00,"37,545",2021-05-29 20:02:13


# 爬取&保存資料

In [8]:
# 建立與db的連線
conn = sqlite3.connect('stock_info.db')
# conn.cursor().execute("drop table top100_stocks")
conn.cursor().execute("drop table stock_info")

In [ ]:
for stockid in top100_stocks['no']:
    try:
        df = get_l24m_stockinfo(stockid=stockid, start='2019-01-01', end='2021-05-29')
        df.to_sql('stock_info', con=conn, if_exists='append', index=False)
        print('Complete:', stockid)
    except:
        print('ERROR: ', stockid)

https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20190101&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20190201&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20190301&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20190401&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20190501&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20190601&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20190701&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20190801&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20190901&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20191001&stockNo=2603
https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20191101&stockNo=2603
https://ww